In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from keras import backend as K
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping


In [4]:
# DATA load
X = pd.read_csv('./content/loan_train_preprocessed.csv')

# backward
X = X[['term', 'initial_list_status', 'int_rate', 
'emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 
'inq_last_6mths', 'revol_util', 'recoveries', 
'collection_recovery_fee', 'tot_cur_bal', 
'home_ownershipRENT', 'purposesmall_business', 
'purposewedding', 'earliest_cr_line2000']]

y = pd.read_csv('./content/loan_train_label.csv')
y = y.drop(['id'], axis=1)

In [5]:
# Dividing the data into train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.20 )
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size = 0.20 )

x_train.shape,y_train.shape,x_val.shape,y_val.shape,x_test.shape,y_test.shape

((10240, 16), (10240, 1), (2560, 16), (2560, 1), (3200, 16), (3200, 1))

In [6]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [7]:

import warnings
warnings.filterwarnings("ignore")



model = Sequential()
model.add(Dense(2048, input_shape = (len(x_train.columns),), activation='ELU'))
model.add(Dense(1024, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='ELU'))
model.add(Dense(1, activation = 'sigmoid'))


model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy', f1])

In [8]:
# Adjust the weights of the classes since your dataset is HIGHLY IMBALANCED!
class_weight = {0: 1.,
                1: 1.}

model.fit(x_train.values,
         y_train.values,
         epochs = 300,
         batch_size = 2048,
         validation_data = (x_val.values, y_val.values), class_weight=class_weight,
         callbacks=[EarlyStopping(monitor='val_f1', mode='max', patience=100, restore_best_weights=True)])


Epoch 1/300
5/5 [==============================] - 2s 212ms/step - loss: 0.6897 - accuracy: 0.5869 - f1: 0.5884 - val_loss: 0.5990 - val_accuracy: 0.6652 - val_f1: 0.6289
Epoch 2/300
5/5 [==============================] - 1s 162ms/step - loss: 0.6146 - accuracy: 0.6465 - f1: 0.6187 - val_loss: 0.5853 - val_accuracy: 0.6727 - val_f1: 0.6494
Epoch 3/300
5/5 [==============================] - 1s 156ms/step - loss: 0.5968 - accuracy: 0.6578 - f1: 0.6409 - val_loss: 0.5722 - val_accuracy: 0.6711 - val_f1: 0.6193
Epoch 4/300
5/5 [==============================] - 1s 160ms/step - loss: 0.5907 - accuracy: 0.6449 - f1: 0.6096 - val_loss: 0.5709 - val_accuracy: 0.6586 - val_f1: 0.6899
Epoch 5/300
5/5 [==============================] - 1s 170ms/step - loss: 0.5800 - accuracy: 0.6657 - f1: 0.6461 - val_loss: 0.5630 - val_accuracy: 0.6859 - val_f1: 0.6649
Epoch 6/300
5/5 [==============================] - 1s 193ms/step - loss: 0.5746 - accuracy: 0.6609 - f1: 0.6323 - val_loss: 0.5631 - val_accuracy

In [9]:
y_prediction = model.predict(x_test.values)
y_prediction= [1 if i>=0.5 else 0 for i in y_prediction]
print("The Test Accuracy of the model is: {} %".format(accuracy_score(y_test.values, y_prediction) * 100.)) 
print()

print(confusion_matrix(y_test.values, y_prediction))
print()

target_names = ['Fully Paid', 'Charged Off']
print(classification_report(y_test, y_prediction, target_names=target_names))

The Test Accuracy of the model is: 68.3125 %

[[1014  570]
 [ 444 1172]]

              precision    recall  f1-score   support

  Fully Paid       0.70      0.64      0.67      1584
 Charged Off       0.67      0.73      0.70      1616

    accuracy                           0.68      3200
   macro avg       0.68      0.68      0.68      3200
weighted avg       0.68      0.68      0.68      3200

